In [0]:
from delta.tables import *

DeltaTable.create(spark)\
    .tableName("employees_demo")\
    .addColumn("ID", "INT")\
    .addColumn("Name", "STRING")\
    .addColumn("Gender", "STRING")\
    .addColumn("Salary", "INT")\
    .addColumn("Department", "STRING")\
    .property("Description", "Schema Evolution Demo")\
    .location("FileStore/tables/delta/path_employees_demo")\
    .execute()

Out[1]: <delta.tables.DeltaTable at 0x7faf90045850>

In [0]:
%sql
SELECT * FROM employees_demo

ID,Name,Gender,Salary,Department


In [0]:
%sql
INSERT INTO employees_demo VALUES(100, "Stephe", "M", 2000, "IT")

num_affected_rows,num_inserted_rows
1,1


In [0]:
_sqldf.display()  # SELECT * FROM employees_demo

ID,Name,Gender,Salary,Department
100,Stephe,M,2000,IT


### Schema Evolution

In [0]:
from pyspark.sql.types import IntegerType, StringType
data = [(200, "Philip", "M", 8000, "HR", "Test Data")]

schema = StructType([StructField("ID", IntegerType(), False),
                    StructField("Name", StringType(), True),
                    StructField("Gender", StringType(), True),
                    StructField("Salary", IntegerType(), True),
                    StructField("Department", StringType(), True),
                    StructField("additionalcol1", StringType(), True)]) # this will alter the original schema

df = spark.createDataFrame(data, schema)
df.display()

ID,Name,Gender,Salary,Department,additionalcol1
200,Philip,M,8000,HR,Test Data


In [0]:
df.write.format("delta").mode("append").saveAsTable("employees_demo")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8121041784125140>:1
----> 1 df.write.format("delta").mode("append").saveAsTable("employees_demo")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1520, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1518 if format is not None:
   1519     self.format(format)
-> 1520 self._jwrite.saveAsTable(name)

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py:1321, in JavaMember.__call__(self, *arg

The above execution will throw due to schema mismatch. Use `mergeSchema` to handle this

In [0]:
df.write.format("delta").option("mergeSchema", True).mode("append").saveAsTable("employees_demo")

In [0]:
%sql
SELECT * FROM employees_demo

ID,Name,Gender,Salary,Department,additionalcol1
200,Philip,M,8000,HR,Test Data
100,Stephe,M,2000,IT,null


In [0]:
data = [(300, "David", "M", 8000, "HR", "Dummy Data")]

schema = StructType([StructField("ID", IntegerType(), False),
                    StructField("Name", StringType(), True),
                    StructField("Gender", StringType(), True),
                    StructField("Salary", IntegerType(), True),
                    StructField("Department", StringType(), True),
                    StructField("additionalcol2", StringType(), True)]) # this will also alter the original schema

df = spark.createDataFrame(data, schema)
df.display()

ID,Name,Gender,Salary,Department,additionalcol2
300,David,M,8000,HR,Dummy Data


In [0]:
df.write.format("delta").option("mergeSchema", True).mode("append").saveAsTable("employees_demo")

In [0]:
%sql
SELECT * FROM employees_demo

ID,Name,Gender,Salary,Department,additionalcol1,additionalcol2
200,Philip,M,8000,HR,Test Data,null
300,David,M,8000,HR,null,Dummy Data
100,Stephe,M,2000,IT,null,null


In [0]:
data = [(400, "Sam", "M", 5000, "Dummy")]

schema = StructType([StructField("ID", IntegerType(), False),
                    StructField("Name", StringType(), True),
                    StructField("Gender", StringType(), True),
                    StructField("Salary", IntegerType(), True),
                    StructField("additionalcol2", StringType(), True)]) # this will also alter the original schema

df = spark.createDataFrame(data, schema)
df.display()

ID,Name,Gender,Salary,additionalcol2
400,Sam,M,5000,Dummy


In [0]:
df.write.format("delta").option("mergeSchema", True).mode("append").saveAsTable("employees_demo")

In [0]:
%sql
SELECT * FROM employees_demo

ID,Name,Gender,Salary,Department,additionalcol1,additionalcol2
200,Philip,M,8000,HR,Test Data,null
300,David,M,8000,HR,null,Dummy Data
400,Sam,M,5000,null,null,Dummy
100,Stephe,M,2000,IT,null,null
